# People to Django
Import SpatiaLite records to a GeoDjango database

### Import libraries, define settings, import model

In [1]:
import os
import django
from django.db import transaction
from django.contrib.gis.geos import GEOSGeometry
import sqlite3
from asgiref.sync import sync_to_async

import sys; sys.path.append('../../')
os.environ['DJANGO_SETTINGS_MODULE'] = 'gaia.settings'
django.setup()

from whale.models import People

### User defined variables

In [2]:
sdb = "C:/gis/gaia/data/databases/gaia.db"

### Perform async insertion to database

In [3]:
@sync_to_async
def insert_people(id, name, email, organization, sub_organization, location):
    """ Insert People into the GeoDjango database"""
    People.objects.create(
        id = id,
        name = name,
        email = email,
        organization = organization,
        sub_organization = sub_organization,
        location = location
    )

async def load_spatialite_data():
    """ Connect to original SpatiaLite Database, select all records
            call insert function in sync_to_async mode.
    """
    conn = sqlite3.connect(sdb)
    conn.enable_load_extension(True)
    conn.execute("SELECT load_extension('mod_spatialite')")
    cursor = conn.cursor()

    cursor.execute("SELECT id, name, email, organization, sub_organization, location from people")

    for row in cursor.fetchall():
        id, name, email, organization, sub_organization, location = row
        await insert_people(id, name, email, organization, sub_organization, location)

    conn.close()

await load_spatialite_data()

# End